# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [58]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-894e89b7d8-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Sherlock Holmes explorative search") 


Consider the following exploratory scenario:


> Investigate the canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes) and which are the films and tv series related to the canon. We are also interested in the Emmy Awards and hiw many awards each film won. 


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q35610` | Arthur Conan Doyle        | node |



Also consider

```
wd:Q35610 ?p ?obj .
```

is the BGP to retrieve all **properties of Arthur Conan Doyle**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes)

2. Identify the BGP for film

3. Identify the BGP for television series

4. Return the year of publication (take the first one if there are more publication dates) for each film and television series related to the canon of Sherlock Holmes (the result set must be a list of triples film/tv series IRI, label and year).

5. Find who acted in more than one film or tv series related to the canon of Sherlock Holmes (the result set must be a list of triples actor/actress IRI, label and #film).

6. Identify the BGP for Primetime Emmy Award

7. Return all the types of Primetime Emmy Award (the result set must be a list of couples award IRI and label).

8. For each film and television series related to the canon of Sherlock Holmes, return the number of Primetime Emmy Awards they won (return also the objects who did not win an Award) (the result set must be a list of triples film/tv series IRI, label and #award).

## Task 1
Identify the BGP for canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes)

In [20]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q35610 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   
   
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4928'), ('name', 'Ricorso author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5714'), ('name', 'Tor.com author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9419'), ('name', 'personal library at')]
[('p', 'http://www.wikidata.org/prop/direct/P5470'), ('name', 'Scottish Poetry Library poet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http:/

In [33]:
# query example
queryString = """
SELECT DISTINCT ?p ?name ?obj ?oname
WHERE {
   # bind something
   wd:Q35610 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname
   
   FILTER(REGEX(?oname ,"canon*"^^xsd:string))
   
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('name', 'notable work'), ('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
1


In [36]:
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]

# query example
queryString = """
SELECT DISTINCT ?p ?name ?obj ?oname
WHERE {
   # bind something
   wd:Q2316684 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'), ('obj', 'http://www.wikidata.org/entity/Q1764019'), ('oname', 'Sherlock Holmes')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q733582'), ('oname', 'His Last Bow')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q45192'), ('oname', 'The Hound of the Baskervilles')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q277759'), ('oname', 'book series')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q860831'), ('oname', 'The Memoirs of Sherlock Holmes')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q1631107'), ('oname', 'bibliography')]
[('p', 'http://www.wikidata.or

Final query for this task

In [39]:
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]

# query example
queryString = """
SELECT DISTINCT ?p ?name ?obj ?oname
WHERE {
   # bind something
   wd:Q2316684 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'), ('obj', 'http://www.wikidata.org/entity/Q1764019'), ('oname', 'Sherlock Holmes')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q733582'), ('oname', 'His Last Bow')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q45192'), ('oname', 'The Hound of the Baskervilles')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q277759'), ('oname', 'book series')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q860831'), ('oname', 'The Memoirs of Sherlock Holmes')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q1631107'), ('oname', 'bibliography')]
[('p', 'http://www.wikidata.or

## Task 2
Identify the BGP for film

In [39]:
# write your queries
# write your final query
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname
WHERE {
   # bind something
   wd:Q35610 wdt:P106 ?obj .
   # get the label
   
   ?obj sc:name ?oname

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11774202'), ('oname', 'essayist')]
[('obj', 'http://www.wikidata.org/entity/Q18844224'), ('oname', 'science fiction writer')]
[('obj', 'http://www.wikidata.org/entity/Q214917'), ('oname', 'playwright')]
[('obj', 'http://www.wikidata.org/entity/Q28389'), ('oname', 'screenwriter')]
[('obj', 'http://www.wikidata.org/entity/Q36180'), ('oname', 'writer')]
[('obj', 'http://www.wikidata.org/entity/Q39631'), ('oname', 'physician')]
[('obj', 'http://www.wikidata.org/entity/Q4853732'), ('oname', "children's writer")]
[('obj', 'http://www.wikidata.org/entity/Q551835'), ('oname', 'physician writer')]
[('obj', 'http://www.wikidata.org/entity/Q6625963'), ('oname', 'novelist')]
[('obj', 'http://www.wikidata.org/entity/Q10297252'), ('oname', 'crime writer')]
10


In [54]:
# write your queries
# write your final query
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname ?s ?pname
WHERE {
   # bind something
   wd:Q35610 ?p ?obj .
   ?obj wdt:P31 ?s.
   # get the label
   
   ?s sc:name ?pname .
   ?obj sc:name ?oname.
   
   FILTER(REGEX(?pname ,"film*"^^xsd:string))
}

LIMIT 300
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q28389'), ('oname', 'screenwriter'), ('s', 'http://www.wikidata.org/entity/Q4220920'), ('pname', 'filmmaking occupation')]
1


In [62]:
# write your queries
# write your final query
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname ?s ?pname
WHERE {
   # bind something
   wd:Q4220920 ?p ?obj .
   ?obj wdt:P31 ?s.
   # get the label
   
   ?s sc:name ?pname .
   ?obj sc:name ?oname.
   
   #FILTER(REGEX(?pname ,"film*"^^xsd:string))
}

LIMIT 300
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/P3092'), ('oname', 'film crew member'), ('s', 'http://www.wikidata.org/entity/Q107395292'), ('pname', 'Wikidata property related to movies and television shows')]
[('obj', 'http://www.wikidata.org/entity/Q1414443'), ('oname', 'filmmaker'), ('s', 'http://www.wikidata.org/entity/Q4220920'), ('pname', 'filmmaking occupation')]
[('obj', 'http://www.wikidata.org/entity/Q1414443'), ('oname', 'filmmaker'), ('s', 'http://www.wikidata.org/entity/Q58635633'), ('pname', 'media profession')]
[('obj', 'http://www.wikidata.org/entity/Q1414443'), ('oname', 'filmmaker'), ('s', 'http://www.wikidata.org/entity/Q47728'), ('pname', 'hobby')]
[('obj', 'http://www.wikidata.org/entity/Q28640'), ('oname', 'profession'), ('s', 'http://www.wikidata.org/entity/Q19361238'), ('pname', 'Wikidata metaclass')]
[('obj', 'http://www.wikidata.org/entity/Q28640'), ('oname', 'profession'), ('s', 'http://www.wikidata.org/entity/Q24017414'), ('pname', 'second-order class')]
[

In [65]:
# write your queries
# write your final query
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname ?s ?pname
WHERE {
   # bind something
   wd:Q2316684 ?p ?obj .
   ?obj wdt:P31 ?s.
   # get the label
   
   ?s sc:name ?pname .
   ?obj sc:name ?oname.
   
}

LIMIT 300
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1764019'), ('oname', 'Sherlock Holmes'), ('s', 'http://www.wikidata.org/entity/Q24856'), ('pname', 'film series')]
[('obj', 'http://www.wikidata.org/entity/Q733582'), ('oname', 'His Last Bow'), ('s', 'http://www.wikidata.org/entity/Q1279564'), ('pname', 'short story collection')]
[('obj', 'http://www.wikidata.org/entity/Q45192'), ('oname', 'The Hound of the Baskervilles'), ('s', 'http://www.wikidata.org/entity/Q17518461'), ('pname', 'posthumous work')]
[('obj', 'http://www.wikidata.org/entity/Q45192'), ('oname', 'The Hound of the Baskervilles'), ('s', 'http://www.wikidata.org/entity/Q7725634'), ('pname', 'literary work')]
[('obj', 'http://www.wikidata.org/entity/Q45192'), ('oname', 'The Hound of the Baskervilles'), ('s', 'http://www.wikidata.org/entity/Q8261'), ('pname', 'novel')]
[('obj', 'http://www.wikidata.org/entity/Q860831'), ('oname', 'The Memoirs of Sherlock Holmes'), ('s', 'http://www.wikidata.org/entity/Q7725634'), ('pname', '

In [64]:
#[('obj', 'http://www.wikidata.org/entity/Q6585139'), ('oname', 'mystery fiction')]
# write your final query
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname ?p ?pname
WHERE {
   # bind something
   wd:Q3406858 ?p ?obj .
   # get the label
   
   ?obj sc:name ?oname.
   ?p sc:name ?pname.

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q702269'), ('oname', 'professional'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('obj', 'http://www.wikidata.org/entity/Q2500638'), ('oname', 'creator'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('obj', 'http://www.wikidata.org/entity/Q4220920'), ('oname', 'filmmaking occupation'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('obj', 'http://www.wikidata.org/entity/Q4220920'), ('oname', 'filmmaking occupation'), ('p', 'http://www.wikidata.org/prop/direct/P425'), ('pname', 'field of this occupation')]
4


In [26]:
# write your queries
# write your final query
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname
WHERE {
   # bind something
   wd:Q2316684 wdt:P136 ?obj .
   # get the label
   
   ?obj sc:name ?oname

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q6585139'), ('oname', 'mystery fiction')]
1


In [32]:
# write your queries
# write your final query
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname ?s ?sname
WHERE {
   # bind something
   ?s wdt:P50 ?obj ;
       sc:sname ?sname.
   # get the label
   
   ?obj sc:name ?oname

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [27]:
#[('obj', 'http://www.wikidata.org/entity/Q6585139'), ('oname', 'mystery fiction')]
# write your final query
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname ?p ?pname
WHERE {
   # bind something
   wd:Q6585139 ?p ?obj .
   # get the label
   
   ?obj sc:name ?oname.
   ?p sc:name ?pname.

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q182015'), ('oname', 'thriller'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('obj', 'http://www.wikidata.org/entity/Q1762165'), ('oname', 'action fiction'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('obj', 'http://www.wikidata.org/entity/Q108465955'), ('oname', 'fiction genre'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('obj', 'http://www.wikidata.org/entity/Q186424'), ('oname', 'detective fiction'), ('p', 'http://www.wikidata.org/prop/direct/P460'), ('pname', 'said to be the same as')]
[('obj', 'http://www.wikidata.org/entity/Q8803649'), ('oname', 'Category:Mystery fiction'), ('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category")]
[('obj', 'http://www.wikidata.org/entity/Q83267'), ('oname', 'crime'), ('p', 'http://www.wikidata.org/prop/direct/P921'), ('pname', 'main subject')]
6


In [33]:
#[('obj', 'http://www.wikidata.org/entity/Q6585139'), ('oname', 'mystery fiction')]
# write your final query
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname ?p ?pname
WHERE {
   # bind something
   wd:Q1762165 ?p ?obj .
   # get the label
   
   ?obj sc:name ?oname.
   ?p sc:name ?pname.

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q8253'), ('oname', 'fiction'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('obj', 'http://www.wikidata.org/entity/Q108465955'), ('oname', 'fiction genre'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('obj', 'http://www.wikidata.org/entity/Q6842826'), ('oname', 'Category:Action (genre)'), ('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category")]
3


In [34]:
#[('obj', 'http://www.wikidata.org/entity/Q6585139'), ('oname', 'mystery fiction')]
# write your final query
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname ?p ?pname
WHERE {
   # bind something
   wd:Q8253 ?p ?obj .
   # get the label
   
   ?obj sc:name ?oname.
   ?p sc:name ?pname.

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19180675'), ('oname', 'Small Brockhaus and Efron Encyclopedic Dictionary'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('obj', 'http://www.wikidata.org/entity/Q602358'), ('oname', 'Brockhaus and Efron Encyclopedic Dictionary'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('obj', 'http://www.wikidata.org/entity/Q87482979'), ('oname', 'work of fiction'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('obj', 'http://www.wikidata.org/entity/Q9134'), ('oname', 'mythology'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('obj', 'http://www.wikidata.org/entity/Q17537576'), ('oname', 'creative work'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('obj', 'http://www.wikidata.org/entity/Q15306849'), ('oname', 'fictitious work'), ('p', 'http://www.wikidat

In [38]:
#[('obj', 'http://www.wikidata.org/entity/Q6585139'), ('oname', 'mystery fiction')]
# write your final query
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname ?p ?pname
WHERE {
   # bind something
   wd:Q16889133 ?p ?obj .
   # get the label
   
   ?obj sc:name ?oname.
   ?p sc:name ?pname.

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5962346'), ('oname', 'classification system'), ('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of')]
[('obj', 'http://www.wikidata.org/entity/Q44325'), ('oname', 'ontology'), ('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of')]
[('obj', 'http://www.wikidata.org/entity/Q7095059'), ('oname', 'ontology language'), ('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of')]
[('obj', 'http://www.wikidata.org/entity/Q217594'), ('oname', 'class'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('obj', 'http://www.wikidata.org/entity/Q37517'), ('oname', 'class'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('obj', 'http://www.wikidata.org/entity/Q99527517'), ('oname', 'collective entity'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('obj', 'http://www.wikidata.org/entity/Q19478619'

In [66]:

#[('obj', 'http://www.wikidata.org/entity/Q6585139'), ('oname', 'mystery fiction')]
# write your final query
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?s ?oname
WHERE {
   # bind something
   ?s wdt:P279 wd:Q1762165 .
   # get the label
   
   ?s sc:name ?oname

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q188473'), ('oname', 'action film')]
[('s', 'http://www.wikidata.org/entity/Q3990883'), ('oname', 'action thriller')]
[('s', 'http://www.wikidata.org/entity/Q15637293'), ('oname', 'action anime and manga')]
[('s', 'http://www.wikidata.org/entity/Q6585139'), ('oname', 'mystery fiction')]
[('s', 'http://www.wikidata.org/entity/Q21802675'), ('oname', 'adventure fiction')]
[('s', 'http://www.wikidata.org/entity/Q11282935'), ('oname', 'action novel')]
[('s', 'http://www.wikidata.org/entity/Q20087918'), ('oname', 'action comics')]
7


In [15]:

#[('obj', 'http://www.wikidata.org/entity/Q6585139'), ('oname', 'mystery fiction')]

#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname
WHERE { 
   # bind something
   wd:Q108465955 wdt:P279 ?obj .
   # get the label
   
   ?obj sc:name ?oname

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1792379'), ('oname', 'art genre')]
1


In [25]:

#[('obj', 'http://www.wikidata.org/entity/Q6585139'), ('oname', 'mystery fiction')]

#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?p ?oname ?s ?sname
WHERE { 
   # bind something
    ?s ?p wd:Q4916;
        sc:name ?sname .
    
   
   # get the label
   
   ?p sc:name ?oname
  

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('oname', 'facet of'), ('s', 'http://www.wikidata.org/entity/Q338071'), ('sname', 'euro banknote')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by'), ('s', 'http://www.wikidata.org/entity/Q16068'), ('sname', 'Deutsche Mark')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by'), ('s', 'http://www.wikidata.org/entity/Q788472'), ('sname', 'Dutch guilder')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by'), ('s', 'http://www.wikidata.org/entity/Q203354'), ('sname', 'Finnish markka')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by'), ('s', 'http://www.wikidata.org/entity/Q270642'), ('sname', 'Irish pound')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by'), ('s', 'http://www.wikidata.org/entity/Q184172'), ('sname', 'French franc')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('oname',

In [5]:

#[('obj', 'http://www.wikidata.org/entity/Q6585139'), ('oname', 'mystery fiction')]

#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname
WHERE {
   # bind something
   wd:Q838948 wdt:P279 ?obj .
   # get the label
   
   ?obj sc:name ?oname

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q17537576'), ('oname', 'creative work')]
[('obj', 'http://www.wikidata.org/entity/Q18593264'), ('oname', 'item of collection or exhibition')]
[('obj', 'http://www.wikidata.org/entity/Q28877'), ('oname', 'goods')]
3


In [68]:
# write your queries

#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
# P31 instance
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname ?s ?pname
WHERE {
   # bind something
   wd:Q2316684 ?p ?obj .
   ?obj wdt:P31 ?s.
   # get the label
   

   ?s sc:name ?pname .
   ?obj sc:name ?oname.

}

LIMIT 300
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1764019'), ('oname', 'Sherlock Holmes'), ('s', 'http://www.wikidata.org/entity/Q24856'), ('pname', 'film series')]
[('obj', 'http://www.wikidata.org/entity/Q733582'), ('oname', 'His Last Bow'), ('s', 'http://www.wikidata.org/entity/Q1279564'), ('pname', 'short story collection')]
[('obj', 'http://www.wikidata.org/entity/Q45192'), ('oname', 'The Hound of the Baskervilles'), ('s', 'http://www.wikidata.org/entity/Q17518461'), ('pname', 'posthumous work')]
[('obj', 'http://www.wikidata.org/entity/Q45192'), ('oname', 'The Hound of the Baskervilles'), ('s', 'http://www.wikidata.org/entity/Q7725634'), ('pname', 'literary work')]
[('obj', 'http://www.wikidata.org/entity/Q45192'), ('oname', 'The Hound of the Baskervilles'), ('s', 'http://www.wikidata.org/entity/Q8261'), ('pname', 'novel')]
[('obj', 'http://www.wikidata.org/entity/Q860831'), ('oname', 'The Memoirs of Sherlock Holmes'), ('s', 'http://www.wikidata.org/entity/Q7725634'), ('pname', '

In [69]:
#('s', 'http://www.wikidata.org/entity/Q11424'), ('pname', 'film')]

#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
# P31 instance
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname ?s ?pname
WHERE {
   # bind something
   wd:Q11424 ?p ?obj .
   
   # get the label

   ?obj sc:name ?oname.

}

LIMIT 300
"""

print("Results")
x=run_query(queryString)

Final query for this task

In [72]:
# write your final query

#('s', 'http://www.wikidata.org/entity/Q11424'), ('pname', 'film')]

#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
# P31 instance
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname 
WHERE {
   # bind something
   ?obj wdt:P31 wd:Q11424.
   
   # get the label

   ?obj sc:name ?oname.

}

LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q8178307'), ('oname', '15 Park Avenue')]
[('obj', 'http://www.wikidata.org/entity/Q8197668'), ('oname', 'Idiot Love')]
[('obj', 'http://www.wikidata.org/entity/Q319263'), ('oname', 'Doctor in the House')]
[('obj', 'http://www.wikidata.org/entity/Q8452990'), ('oname', "Sunday League - Pepik Hnatek's Final Match")]
[('obj', 'http://www.wikidata.org/entity/Q284184'), ('oname', 'Ra.One')]
[('obj', 'http://www.wikidata.org/entity/Q13637926'), ('oname', 'Indra')]
[('obj', 'http://www.wikidata.org/entity/Q13638153'), ('oname', 'Inner Senses')]
[('obj', 'http://www.wikidata.org/entity/Q13645807'), ('oname', 'Madhumasam')]
[('obj', 'http://www.wikidata.org/entity/Q13647258'), ('oname', 'Marupakkam')]
[('obj', 'http://www.wikidata.org/entity/Q13677566'), ('oname', 'Timberjack')]
10


## Task 3
Identify the BGP for television series

In [60]:
# write your queries
#[('obj', 'http://www.wikidata.org/entity/Q6585139'), ('oname', 'mystery fiction')]
# write your final query
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname ?s ?pname ?m ?n ?tpname ?toname
WHERE {
   # bind something
   ?s wdt:P136 ?obj .
   # get the label
   ?obj ?m ?n.
   
   ?m sc:name ?tpname.
   ?n sc:name ?toname.
   
   ?obj sc:name ?oname.
   ?s sc:name ?pname.
   
   FILTER(REGEX(?oname ,"television series"^^xsd:string)) 
   

}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q84270297'), ('oname', 'romance television series'), ('s', 'http://www.wikidata.org/entity/Q491376'), ('pname', 'Get Karl! Oh Soo-jung'), ('m', 'http://www.wikidata.org/prop/direct/P279'), ('n', 'http://www.wikidata.org/entity/Q19765983'), ('tpname', 'subclass of'), ('toname', 'romantic fiction')]
[('obj', 'http://www.wikidata.org/entity/Q84270297'), ('oname', 'romance television series'), ('s', 'http://www.wikidata.org/entity/Q491683'), ('pname', 'To the Beautiful You'), ('m', 'http://www.wikidata.org/prop/direct/P279'), ('n', 'http://www.wikidata.org/entity/Q19765983'), ('tpname', 'subclass of'), ('toname', 'romantic fiction')]
[('obj', 'http://www.wikidata.org/entity/Q84270297'), ('oname', 'romance television series'), ('s', 'http://www.wikidata.org/entity/Q2656704'), ('pname', 'Lost in Austen'), ('m', 'http://www.wikidata.org/prop/direct/P279'), ('n', 'http://www.wikidata.org/entity/Q19765983'), ('tpname', 'subclass of'), ('toname', 

Final query for this task

In [3]:
# write your final query
# write your queries
#[('obj', 'http://www.wikidata.org/entity/Q6585139'), ('oname', 'mystery fiction')]
# write your final query
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')
# query example
queryString = """
SELECT DISTINCT  ?obj ?oname ?s ?pname ?m ?n ?tpname ?toname
WHERE {
   # bind something
   ?s wdt:P136 ?obj .
   # get the label
   ?obj ?m ?n.
   
   ?m sc:name ?tpname.
   ?n sc:name ?toname.
   
   ?obj sc:name ?oname.
   ?s sc:name ?pname.
   
   FILTER(REGEX(?oname ,"television series"^^xsd:string)) 
   

}

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q84270297'), ('oname', 'romance television series'), ('s', 'http://www.wikidata.org/entity/Q491376'), ('pname', 'Get Karl! Oh Soo-jung'), ('m', 'http://www.wikidata.org/prop/direct/P279'), ('n', 'http://www.wikidata.org/entity/Q19765983'), ('tpname', 'subclass of'), ('toname', 'romantic fiction')]
[('obj', 'http://www.wikidata.org/entity/Q84270297'), ('oname', 'romance television series'), ('s', 'http://www.wikidata.org/entity/Q491683'), ('pname', 'To the Beautiful You'), ('m', 'http://www.wikidata.org/prop/direct/P279'), ('n', 'http://www.wikidata.org/entity/Q19765983'), ('tpname', 'subclass of'), ('toname', 'romantic fiction')]
[('obj', 'http://www.wikidata.org/entity/Q84270297'), ('oname', 'romance television series'), ('s', 'http://www.wikidata.org/entity/Q2656704'), ('pname', 'Lost in Austen'), ('m', 'http://www.wikidata.org/prop/direct/P279'), ('n', 'http://www.wikidata.org/entity/Q19765983'), ('tpname', 'subclass of'), ('toname', 

## Task 4
Return the year of publication (take the first one if there are more publication dates) for each film and television series related to the canon of Sherlock Holmes (the result set must be a list of triples film/tv series IRI, label and year).

In [86]:
# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]

# query example
queryString = """
SELECT DISTINCT ?p ?name ?obj ?oname
WHERE {
   # bind something
   wd:Q2316684 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname

}

LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'), ('obj', 'http://www.wikidata.org/entity/Q1764019'), ('oname', 'Sherlock Holmes')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q733582'), ('oname', 'His Last Bow')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q45192'), ('oname', 'The Hound of the Baskervilles')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q277759'), ('oname', 'book series')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q860831'), ('oname', 'The Memoirs of Sherlock Holmes')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q1631107'), ('oname', 'bibliography')]
[('p', 'http://www.wikidata.or

In [5]:
#[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'),

# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), 
#('obj', 'http://www.wikidata.org/entity/Q53815'), ('oname', 'canon')]

# query example
queryString = """
SELECT DISTINCT  ?p ?movp
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?obj .
   
   ?obj wdt:P31 ?pro;
           ?p ?o.
   # get the label
   ?p sc:name ?movp.

}


LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('movp', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('movp', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('movp', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('movp', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('movp', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('movp', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('movp', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('movp', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('movp', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('movp', 'based on')]
10


In [17]:
#[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'),

# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), 
#('obj', 'http://www.wikidata.org/entity/Q53815'), ('oname', 'canon')]

# query example
queryString = """
SELECT DISTINCT ?obj ?pro
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?obj .
   ?obj wdt:P31 ?oy.
   
   ?obj sc:name ?pro.
   VALUES ?oy {wd:Q5398426 wd:Q11424}
        
}


LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q192837'), ('pro', 'Sherlock')]
[('obj', 'http://www.wikidata.org/entity/Q247643'), ('pro', 'Elementary')]
[('obj', 'http://www.wikidata.org/entity/Q4523158'), ('pro', 'Sherlock Holmes')]
[('obj', 'http://www.wikidata.org/entity/Q200396'), ('pro', 'Sherlock Holmes')]
[('obj', 'http://www.wikidata.org/entity/Q387306'), ('pro', 'Sherlock Holmes')]
[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows')]
6


In [34]:
#[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'),

# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), 
#('obj', 'http://www.wikidata.org/entity/Q53815'), ('oname', 'canon')]
#[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows')]
# query example
queryString = """
SELECT DISTINCT ?p ?pname ?obj
WHERE {
   # bind something
   wd:Q243643 ?p ?obj .
   
   #?obj sc:name ?pro.
   ?p sc:name ?pname.
        
}


LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pname', 'film editor'), ('obj', 'http://www.wikidata.org/entity/Q1110463')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('pname', 'Box Office Mojo film ID (former scheme)'), ('obj', 'sherlockholmes2')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pname', 'Rotten Tomatoes ID'), ('obj', 'm/sherlock_holmes_a_game_of_shadows')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('pname', 'AlloCiné film ID'), ('obj', '173048')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('obj', 'http://www.wikidata.org/entity/Q188473')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('obj', 'http://www.wikidata.org/entity/Q52162262')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('obj', 'http://www.wikidata.org/entity/Q1200678')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('obj', 'http://www.wikidata.org/entity/Q3192

In [44]:
#[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'),

# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), 
#('obj', 'http://www.wikidata.org/entity/Q53815'), ('oname', 'canon')]
#[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows')]
#[('p', 'http://www.wikidata.org/prop/direct/P577'), ('pname', 'publication date'), ('obj', '2011-12-16T00:00:00Z')]
# query example
queryString = """
SELECT DISTINCT ?obj ?pname (MIN(year(?k)) as ?publication_year)
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?obj.
   ?obj wdt:P31 ?h;
        wdt:P577 ?k.
   
   #?obj sc:name ?pro.
   ?obj sc:name ?pname.
   
   VALUES ?h {wd:Q5398426 wd:Q11424}
        
}


LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q200396'), ('pname', 'Sherlock Holmes'), ('publication_year', '2009')]
[('obj', 'http://www.wikidata.org/entity/Q247643'), ('pname', 'Elementary'), ('publication_year', '2013')]
[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pname', 'Sherlock Holmes: A Game of Shadows'), ('publication_year', '2011')]
3


Final query for this task

In [59]:
# write your final query
#[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'),

# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), 
#('obj', 'http://www.wikidata.org/entity/Q53815'), ('oname', 'canon')]
#[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows')]
#[('p', 'http://www.wikidata.org/prop/direct/P577'), ('pname', 'publication date'), ('obj', '2011-12-16T00:00:00Z')]
# query example
queryString = """
SELECT DISTINCT ?obj ?pname (MIN(year(?k)) as ?publication_year)
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?obj.
   ?obj wdt:P31 ?h;
        wdt:P577 ?k.
   
   #?obj sc:name ?pro.
   ?obj sc:name ?pname.
   
   VALUES ?h {wd:Q5398426 wd:Q11424}
        
}


LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q200396'), ('pname', 'Sherlock Holmes'), ('publication_year', '2009')]
[('obj', 'http://www.wikidata.org/entity/Q247643'), ('pname', 'Elementary'), ('publication_year', '2013')]
[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pname', 'Sherlock Holmes: A Game of Shadows'), ('publication_year', '2011')]
3


## Task 5
Find who acted in more than one film or tv series related to the canon of Sherlock Holmes (the result set must be a list of triples actor/actress IRI, label and #film).

In [63]:
# write your queries
#[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'),

# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), 
#('obj', 'http://www.wikidata.org/entity/Q53815'), ('oname', 'canon')]
#[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows')]
#[('p', 'http://www.wikidata.org/prop/direct/P674'), ('pname', 'characters')]
# query example
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {{
   # bind something
   wd:Q243643 ?p ?obj .
   
   #?obj sc:name ?pro.
   ?p sc:name ?pname.
   }
   UNION {
   ?s ?p wd:Q243643.
   ?p sc:name ?pname.
   
   }
        
}


LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pname', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('pname', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pname', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('pname', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('pname', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pname', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('pname', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('pname', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('pname', 'Metacritic ID')]
[('p', 'http://www.wiki

In [ ]:
#[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]
# write your queries
#[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'),

# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), 
#('obj', 'http://www.wikidata.org/entity/Q53815'), ('oname', 'canon')]
#[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows')]
#[('p', 'http://www.wikidata.org/prop/direct/P674'), ('pname', 'characters')]
# query example
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {{
   # bind something
   wd:Q243643 ?p ?obj .
   
   #?obj sc:name ?pro.
   ?p sc:name ?pname.
   }
   UNION {
   ?s ?p wd:Q243643.
   ?p sc:name ?pname.
   
   }
        
}


LIMIT 100
"""

print("Results")
x=run_query(queryString)



In [53]:
# write your queries
#[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'),

# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), 
#('obj', 'http://www.wikidata.org/entity/Q53815'), ('oname', 'canon')]
#[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows')]
#[('p', 'http://www.wikidata.org/prop/direct/P674'), ('pname', 'characters')]
# query example
queryString = """
SELECT DISTINCT ?obj ?pname
WHERE {
   # bind something
   wd:Q2316684 wdt:P674 ?obj .
   
   #?obj sc:name ?pro.
   ?obj sc:name ?pname.
        
}


LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q1067723'), ('pname', 'Inspector Lestrade')]
[('obj', 'http://www.wikidata.org/entity/Q187349'), ('pname', 'Dr. John Watson')]
[('obj', 'http://www.wikidata.org/entity/Q4653'), ('pname', 'Sherlock Holmes')]
[('obj', 'http://www.wikidata.org/entity/Q283111'), ('pname', 'Professor Moriarty')]
[('obj', 'http://www.wikidata.org/entity/Q2456753'), ('pname', 'Mrs. Hudson')]
[('obj', 'http://www.wikidata.org/entity/Q1616457'), ('pname', 'Mycroft Holmes')]
6


In [55]:
# write your final query
#[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'),

# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), 
#('obj', 'http://www.wikidata.org/entity/Q53815'), ('oname', 'canon')]
#[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows')]
#[('p', 'http://www.wikidata.org/prop/direct/P577'), ('pname', 'publication date'), ('obj', '2011-12-16T00:00:00Z')]
# query example
queryString = """
SELECT DISTINCT ?obj ?pname 
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?obj.
   ?obj wdt:P31 ?h;
        wdt:P577 ?k.
   
   #?obj sc:name ?pro.
   ?obj sc:name ?pname.
   
        
}


LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q247643'), ('pname', 'Elementary')]
[('obj', 'http://www.wikidata.org/entity/Q200396'), ('pname', 'Sherlock Holmes')]
[('obj', 'http://www.wikidata.org/entity/Q1210893'), ('pname', 'The Adventures of Sherlock Holmes and Dr. Watson')]
[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pname', 'Sherlock Holmes: A Game of Shadows')]
4


In [83]:

#[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'),
#[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]

# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), 
#('obj', 'http://www.wikidata.org/entity/Q53815'), ('oname', 'canon')]
#[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows')]
# query example
queryString = """
SELECT DISTINCT ?k ?cast_name (COUNT(DISTINCT ?obj) as ?no_of_film)
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?obj.
   ?obj wdt:P31 ?h;
        wdt:P161 ?k.
   
   #?obj sc:name ?pro.
   ?k sc:name ?cast_name.
   
   VALUES ?h {wd:Q5398426 wd:Q11424}
        
}
GROUP BY ?k ?cast_name
HAVING (COUNT (DISTINCT ?obj) > 1)


LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('k', 'http://www.wikidata.org/entity/Q1102002'), ('cast_name', 'Clive Russell'), ('no_of_film', '2')]
[('k', 'http://www.wikidata.org/entity/Q190386'), ('cast_name', 'Rachel McAdams'), ('no_of_film', '2')]
[('k', 'http://www.wikidata.org/entity/Q45647'), ('cast_name', 'Eddie Marsan'), ('no_of_film', '2')]
[('k', 'http://www.wikidata.org/entity/Q440926'), ('cast_name', 'Geraldine James'), ('no_of_film', '2')]
[('k', 'http://www.wikidata.org/entity/Q165219'), ('cast_name', 'Robert Downey Jr.'), ('no_of_film', '2')]
[('k', 'http://www.wikidata.org/entity/Q160432'), ('cast_name', 'Jude Law'), ('no_of_film', '2')]
[('k', 'http://www.wikidata.org/entity/Q236167'), ('cast_name', 'Kelly Reilly'), ('no_of_film', '2')]
7


Final query for this task

In [84]:
# write your final query
#[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'),
#[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]

# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), 
#('obj', 'http://www.wikidata.org/entity/Q53815'), ('oname', 'canon')]
#[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows')]
# query example
queryString = """
SELECT DISTINCT ?k ?cast_name (COUNT(DISTINCT ?obj) as ?no_of_film)
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?obj.
   ?obj wdt:P31 ?h;
        wdt:P161 ?k.
   
   #?obj sc:name ?pro.
   ?k sc:name ?cast_name.
   
   VALUES ?h {wd:Q5398426 wd:Q11424}
        
}
GROUP BY ?k ?cast_name
HAVING (COUNT (DISTINCT ?obj) > 1)


LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('k', 'http://www.wikidata.org/entity/Q1102002'), ('cast_name', 'Clive Russell'), ('no_of_film', '2')]
[('k', 'http://www.wikidata.org/entity/Q190386'), ('cast_name', 'Rachel McAdams'), ('no_of_film', '2')]
[('k', 'http://www.wikidata.org/entity/Q45647'), ('cast_name', 'Eddie Marsan'), ('no_of_film', '2')]
[('k', 'http://www.wikidata.org/entity/Q440926'), ('cast_name', 'Geraldine James'), ('no_of_film', '2')]
[('k', 'http://www.wikidata.org/entity/Q165219'), ('cast_name', 'Robert Downey Jr.'), ('no_of_film', '2')]
[('k', 'http://www.wikidata.org/entity/Q160432'), ('cast_name', 'Jude Law'), ('no_of_film', '2')]
[('k', 'http://www.wikidata.org/entity/Q236167'), ('cast_name', 'Kelly Reilly'), ('no_of_film', '2')]
7


## Task 6
Identify the BGP for Primetime Emmy Award

In [93]:
# write your queries

#[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'),
#[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]

# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), 
#('obj', 'http://www.wikidata.org/entity/Q53815'), ('oname', 'canon')]
#[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows')]
# query example
queryString = """
SELECT DISTINCT ?k ?cast_name ?t ?t_name
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?obj.
   ?obj wdt:P31 ?h;
        ?t ?k.
   
   #?obj sc:name ?pro.
   ?k sc:name ?cast_name.
   ?t sc:name ?t_name.
   
   VALUES ?h {wd:Q5398426 wd:Q11424}
        
}



LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('k', 'http://www.wikidata.org/entity/Q22963868'), ('cast_name', 'Vanessa Aspillaga'), ('t', 'http://www.wikidata.org/prop/direct/P161'), ('t_name', 'cast member')]
[('k', 'http://www.wikidata.org/entity/Q391359'), ('cast_name', 'David Harbour'), ('t', 'http://www.wikidata.org/prop/direct/P161'), ('t_name', 'cast member')]
[('k', 'http://www.wikidata.org/entity/Q60493'), ('cast_name', 'Lee Tergesen'), ('t', 'http://www.wikidata.org/prop/direct/P161'), ('t_name', 'cast member')]
[('k', 'http://www.wikidata.org/entity/Q164131'), ('cast_name', 'John Polson'), ('t', 'http://www.wikidata.org/prop/direct/P162'), ('t_name', 'producer')]
[('k', 'http://www.wikidata.org/entity/Q198684'), ('cast_name', 'Wayne Duvall'), ('t', 'http://www.wikidata.org/prop/direct/P161'), ('t_name', 'cast member')]
[('k', 'http://www.wikidata.org/entity/Q275658'), ('cast_name', 'Wade Williams'), ('t', 'http://www.wikidata.org/prop/direct/P161'), ('t_name', 'cast member')]
[('k', 'http://www.wikidata.org/en

Final query for this task

In [99]:
# write your final query
#[('k', 'http://www.wikidata.org/entity/Q7243506'), ('cast_name', 'Primetime Emmy Award for Outstanding Made for Television Movie')

# write your queries

#[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'),
#[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]

# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), 
#('obj', 'http://www.wikidata.org/entity/Q53815'), ('oname', 'canon')]
#[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows')]
# query example
queryString = """
SELECT DISTINCT ?obj ?pname
WHERE {{
   # bind something
   wd:Q7243506 ?p ?obj .   
   ?obj sc:name ?pname.  
   }
   UNION {
   ?obj ?p wd:Q7243506.
   ?obj sc:name ?pname.
   
   }
        
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q30'), ('pname', 'United States of America')]
[('obj', 'http://www.wikidata.org/entity/Q1044427'), ('pname', 'Primetime Emmy Award')]
[('obj', 'http://www.wikidata.org/entity/Q38033430'), ('pname', 'class of award')]
[('obj', 'http://www.wikidata.org/entity/Q558112'), ('pname', 'Black Mirror')]
[('obj', 'http://www.wikidata.org/entity/Q229305'), ('pname', 'Joan Allen')]
[('obj', 'http://www.wikidata.org/entity/Q1159206'), ('pname', 'Dana Brunetti')]
[('obj', 'http://www.wikidata.org/entity/Q1033016'), ('pname', 'Halle Berry')]
[('obj', 'http://www.wikidata.org/entity/Q300341'), ('pname', 'A Bright Shining Lie')]
[('obj', 'http://www.wikidata.org/entity/Q560286'), ('pname', 'Mark Gatiss')]
[('obj', 'http://www.wikidata.org/entity/Q3133366'), ('pname', 'Sue Vertue')]
[('obj', 'http://www.wikidata.org/entity/Q764812'), ('pname', 'August Wilson')]
[('obj', 'http://www.wikidata.org/entity/Q315808'), ('pname', 'Neil Simon')]
[('obj', 'http://w

## Task 7
Return all the types of Primetime Emmy Award (the result set must be a list of couples award IRI and label).

In [104]:
# write your queries
#[('obj', 'http://www.wikidata.org/entity/Q1044427'), ('pname', 'Primetime Emmy Award')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')

queryString = """
SELECT DISTINCT ?a ?pname 
WHERE {
   # bind something
   ?a wdt:P31 wd:Q1044427.
   ?a sc:name ?pname.     
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q18153303'), ('pname', 'Primetime Emmy Award for Outstanding Unstructured Reality Program')]
[('a', 'http://www.wikidata.org/entity/Q28445729'), ('pname', 'Primetime Emmy Award for Outstanding Actress in a Short Form Comedy or Drama Series')]
[('a', 'http://www.wikidata.org/entity/Q989439'), ('pname', 'Primetime Emmy Award for Outstanding Lead Actor in a Drama Series')]
[('a', 'http://www.wikidata.org/entity/Q2604800'), ('pname', 'Primetime Emmy Award for Outstanding Guest Actor in a Drama Series')]
[('a', 'http://www.wikidata.org/entity/Q7243506'), ('pname', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('a', 'http://www.wikidata.org/entity/Q1277961'), ('pname', 'Primetime Emmy Award for Outstanding Directing for a Comedy Series')]
[('a', 'http://www.wikidata.org/entity/Q3123491'), ('pname', 'Primetime Emmy Award for Outstanding Writing for a Drama Series')]
[('a', 'http://www.wikidata.org/entity/Q20714679'), ('pname

Final query for this task

In [105]:
# write your final query
# write your queries
#[('obj', 'http://www.wikidata.org/entity/Q1044427'), ('pname', 'Primetime Emmy Award')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')

queryString = """
SELECT DISTINCT ?a ?pname 
WHERE {
   # bind something
   ?a wdt:P31 wd:Q1044427.
   ?a sc:name ?pname.     
}

LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q18153303'), ('pname', 'Primetime Emmy Award for Outstanding Unstructured Reality Program')]
[('a', 'http://www.wikidata.org/entity/Q28445729'), ('pname', 'Primetime Emmy Award for Outstanding Actress in a Short Form Comedy or Drama Series')]
[('a', 'http://www.wikidata.org/entity/Q989439'), ('pname', 'Primetime Emmy Award for Outstanding Lead Actor in a Drama Series')]
[('a', 'http://www.wikidata.org/entity/Q2604800'), ('pname', 'Primetime Emmy Award for Outstanding Guest Actor in a Drama Series')]
[('a', 'http://www.wikidata.org/entity/Q7243506'), ('pname', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('a', 'http://www.wikidata.org/entity/Q1277961'), ('pname', 'Primetime Emmy Award for Outstanding Directing for a Comedy Series')]
[('a', 'http://www.wikidata.org/entity/Q3123491'), ('pname', 'Primetime Emmy Award for Outstanding Writing for a Drama Series')]
[('a', 'http://www.wikidata.org/entity/Q20714679'), ('pname

## Task 8
For each film and television series related to the canon of Sherlock Holmes, return the number of Primetime Emmy Awards they won (return also the objects who did not win an Award) (the result set must be a list of triples film/tv series IRI, label and #award).

In [122]:


#[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'),
#[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]

# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), 
#('obj', 'http://www.wikidata.org/entity/Q53815'), ('oname', 'canon')]
#[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows')]
# query example
queryString = """
SELECT DISTINCT ?obj ?pro  (Count(DISTINCT ?k) as ?awards_win)
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?obj.
   ?obj wdt:P31 ?h.
   
   OPTIONAL {
    ?obj wdt:P166 ?k.
    ?k wdt:P31 wd:Q1044427.  
   }
   
   VALUES ?h {wd:Q5398426 wd:Q11424} 
   ?obj sc:name ?pro.
}


LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q387306'), ('pro', 'Sherlock Holmes'), ('awards_win', '0')]
[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows'), ('awards_win', '0')]
[('obj', 'http://www.wikidata.org/entity/Q200396'), ('pro', 'Sherlock Holmes'), ('awards_win', '0')]
[('obj', 'http://www.wikidata.org/entity/Q192837'), ('pro', 'Sherlock'), ('awards_win', '8')]
[('obj', 'http://www.wikidata.org/entity/Q247643'), ('pro', 'Elementary'), ('awards_win', '0')]
[('obj', 'http://www.wikidata.org/entity/Q4523158'), ('pro', 'Sherlock Holmes'), ('awards_win', '0')]
6


Final query for this task

In [123]:
# write your final query


#[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'),
#[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]

# write your queries
#('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oname', 'canon of Sherlock Holmes')]
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), 
#('obj', 'http://www.wikidata.org/entity/Q53815'), ('oname', 'canon')]
#[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows')]
# query example
queryString = """
SELECT DISTINCT ?obj ?pro  (Count(DISTINCT ?k) as ?awards_win)
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?obj.
   ?obj wdt:P31 ?h.
   
   OPTIONAL {
    ?obj wdt:P166 ?k.
    ?k wdt:P31 wd:Q1044427.  
   }
   
   VALUES ?h {wd:Q5398426 wd:Q11424} 
   ?obj sc:name ?pro.
}


LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q387306'), ('pro', 'Sherlock Holmes'), ('awards_win', '0')]
[('obj', 'http://www.wikidata.org/entity/Q243643'), ('pro', 'Sherlock Holmes: A Game of Shadows'), ('awards_win', '0')]
[('obj', 'http://www.wikidata.org/entity/Q200396'), ('pro', 'Sherlock Holmes'), ('awards_win', '0')]
[('obj', 'http://www.wikidata.org/entity/Q192837'), ('pro', 'Sherlock'), ('awards_win', '8')]
[('obj', 'http://www.wikidata.org/entity/Q247643'), ('pro', 'Elementary'), ('awards_win', '0')]
[('obj', 'http://www.wikidata.org/entity/Q4523158'), ('pro', 'Sherlock Holmes'), ('awards_win', '0')]
6
